#Setup environment

In [ ]:
#Clone DALF repo
%cd /content
!git clone 'https://github.com/verlab/DALF_CVPR_2023.git'
%cd /content/DALF_CVPR_2023
import sys
sys.path.append('/content/DALF_CVPR_2023')

#Install kornia dependency
!pip install kornia

# Get datasets from 1DSfM project
In this example we use only a fraction of the images, please uncomment the last line to obtain all images, it should more time (about 10-15 minutes).

In [ ]:
from modules.dataset import getdata

#Get Sample dataset to be faster
getdata.retrieve(data_path = './dataset', reduced = True)

#If you want FULL DATASET, PLEASE USE THE BELOW COMMAND TO OBTAIN ALL DATASETS
#getdata.retrieve(data_path = './nonrigid-keypoint-detector-descriptor/dataset', reduced = False)

# Train the network
Please notice that training entirely on Colab would require **more than a day** of training with the free GPU tier. You can train locally with your GPU.

In [ ]:
# Load the TensorBoard notebook extension to monitor our training
!mkdir './logdir'
%load_ext tensorboard
%tensorboard --logdir "./logdir"

## First-stage training
Here, we train the backbone and detector, without the deformation-aware module. Please refer to the [paper](https://arxiv.org/abs/2304.00583) for more details.

In [ ]:
#Now lets train first stage!
! python3 train.py --mode "ts1" --datapath "./dataset/*/images/*.jpg" --logdir "./logdir" --save "./logdir" #--dry_run

## Second-stage training
In second stage, backbone is frozen and WarperNet alongside the detector decoder are optimized to refine detections and account for strong deformations. The fusion module is also optimized to optimally fuse the distinct and invariant descriptors.

In [ ]:
#Now lets train second stage!

saved_model="./logdir/model_ts1_80000_final.pth"

#saved_model="./logdir/model_ts1_1000_final.pth" #Used as a quick test with --dry_run in the first training stage

! python3 train.py --mode "ts-fl" --datapath "./dataset/*/images/*.jpg" --logdir "./logdir" --pretrained $saved_model --save "./logdir"